In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic = False)


wt.case_study_trains.train = {"case_study": "solaire",
                             "reference": "nawi",
                             "scenario": "baseline"}

wt.case_study_trains.source_water = {"case_study": "solaire", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": ["solaire_graywater", "solaire_blackwater", "solaire_stormwater"]}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.



solaire
------- Adding Unit Processes -------
gray_black
stormwater
treated_storage
10800.0
water_pumping_station
mbr_nitrification
uv_aop
ozone_aop
media_filtration
cas
microfiltration
agriculture
municipal_drinking
surface_discharge
-------------------------------------
adding source
adding source
adding source
adding splitter
----- Connecting Unit Processes -----
solaire_stormwater ToUnitName --> stormwater
gray_black ToUnitName --> water_pumping_station
stormwater ToUnitName --> media_filtration
water_pumping_station ToUnitName --> mbr_nitrification
mbr_nitrification ToUnitName --> uv_aop
uv_aop ToUnitName --> ozone_aop
media_filtration ToUnitName --> cas
cas ToUnitName --> microfiltration
solaire_graywater ToUnitName --> mixer1
solaire_blackwater ToUnitName --> mixer1
mixer1 ToUnitName --> gray_black
ozone_aop ToUnitName --> mixer2
microfiltration ToUnitName --> mixer2
mixer2 ToUnitName --> treated_storage
splitter1 ToUnitName --> agriculture
splitter1 ToUnitName --> municipal_dri

In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [4]:
# Readjust recovery constraint and deactivate objective constraint
# m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)
# m.fs.objective_function.deactivate()

In [5]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=True, skip_small = False)
wt.run_water_tap(m = m, objective=False, print_model_results="summary", skip_small=False)

initial inflow to train is relatively small (< 1 m3/s). running model with dummy flows to initialize.
degrees_of_freedom: 0
WaterTAP3 solver returned an infeasible solution
Running again with updated initial conditions --- attempt 1
WaterTAP3 solution maxIterations
model finished running to initialize conditions. now running with actual inflows.
degrees_of_freedom: 0


ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])

In [ ]:
import numpy as np
import pandas as pd


def it_gets_results(appended='1', print_it=True, write_it=False):    
    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost",
    "flow_in", "flow_out"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']

    case_study = train['case_study']
    if print_it:
        print(f'\n######### Case study = {case_study} #########\n\n\n')

    cap_inv_tot = []
# 
    big_dict = {}

    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            if print_it:
                print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            df = pd.read_csv("data/case_study_results.csv")
            df = df[df.case_study == train['case_study']]
            df = df[df.scenario == train['scenario']]
            df = df[df.unit_process == unit]
            flow_in_excel = round(df[df.variable == "flow_vol_in"].value.sum(), 5)
            flow_out_excel = round(df[df.variable == 'flow_vol_out'].value.sum(), 5)
            flow_in_python = round(value(b_unit.inlet.flow_vol[0.0] * 3600), 5)
            flow_out_python = round(value(b_unit.outlet.flow_vol[0.0] * 3600), 5)
            waste_python = round(value(b_unit.waste.flow_vol[0.0] * 3600), 5)
            if print_it:
                print(f'\n#### flow_in\nPython --> {flow_in_python}')
                print(f'Excel --> {flow_in_excel}\n\n')
                print(f'#### flow out\nPython --> {flow_out_python}')
                print(f'Excel --> {flow_out_excel}\n\n')
                print(f'#### waste\nPython--> {waste_python}\n')
            for py, ex in dict(zip(py_var, excel_var)).items():
                if print_it:
                    print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    if print_it:
                        print(f'Python --> {round(num, 5)}')
                except ZeroDivisionError:
                    if print_it:
                        print(f'Python --> {0} - ERROR')
                    py_vals.append(0)

                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.variable_longform == ex].value.sum()
                if print_it:
                    print(f'Excel --> {round(num, 5)}\n\n')
                excel_vals.append(num)
            py_vals.append(flow_in_python)
            py_vals.append(flow_out_python)
            py_dict = dict(zip(py_var, py_vals))
            excel_vals.append(flow_in_excel)
            excel_vals.append(flow_out_excel)
            excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict

    else:
        pass 

    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                               orient='index')

    df['case_study'] = case_study
    df['LCOW'] = value(m.fs.costing.LCOW)
    if write_it:
        df.to_csv(f'results/compare_{case_study}_{appended}.csv')

In [ ]:
it_gets_results()